## Player Types:

- `zs`: An AI agent initialized with a zero-shot prompt. Just the gave description.
- `spp`: Solo Performance Prompting; an AI agent initialized with the SPP prompt.
- `cot`: Chain-of-Thought; an AI agent initialized with the COT prompt.
- `srep`: Singe-Round-Equilibrium-Player; a player who strictly follows the Single Round Equilibrium Strategy (a specific probability distribution over the available moves)
- `pp`: Pattern Player; Follows a cyclic pattern of moves. Always playes moves from this pattern.
- `ap`: Adaptive Player; finds the most frequent move their opponent plays and counters it.
- `tft`: Tit-for-Tat Player; counters opponent's last played move.

## SC experiments:

- We compare an AI agent in a SC environment vs all other agents. If the opponent is also an AI agent, then the opponent in **NOT** a SC player.
- SC: Each time the AI agent has to play. They generate 5 different answers. We then choose the most frequent result and choose an answer that gave that result. We continue with this history for the rest of the game (as many rounds as it is). Conficts are resolved at random and/or by choosing the first answer that gave the result we picked.

In [1]:
models = [
    {
        "id" : "anthropic.claude-3-5-sonnet-20241022-v2:0",
        "name" : "Claude 3.5 Sonnet v2",
        "thinking" : False,
    },
    {
        "id" : "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        "name" : "Claude 3.7 Sonnet",
        "thinking" : False,
    },
    {
        "id" : "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        "name" : "Claude 3.7 Sonnet (Thinking)",
        "thinking" : True,
    },
    {
        "id" : "us.anthropic.claude-sonnet-4-20250514-v1:0",
        "name" : "Claude Sonnet 4",
        "thinking" : False,
    },
    {
        "id" : "us.anthropic.claude-sonnet-4-20250514-v1:0",
        "name" : "Claude Sonnet 4 (Thinking)",
        "thinking" : True,
    },
    {
        "id" : "us.meta.llama3-3-70b-instruct-v1:0",
        "name" : "Llama 3.3 70B Instruct",
        "thinking" : False,
    },
    {
        "id" : "mistral.mistral-large-2407-v1:0",
        "name" : "Mistral Large (24.07)",
        "thinking" : False,
    },
    {
        "id" : "us.deepseek.r1-v1:0",
        "name" : "DeepSeek-R1",
        "thinking" : False,
    },
]

game_settings_types = ["eq1", "eq1-alt", "ba3", "ba3-alt"]

prompt_types = ["default", "spp", "cot"]

In [2]:
import pandas as pd
import os
import json
from collections import defaultdict

def get_total_points_dataframe(
    log_dir: str,
    model_names: list[str],
    prompt_types: list[str],
    game_type: str,
    game_settings_type: str,
    iteration_cnt: int,
    tot: bool,
) -> pd.DataFrame:
    y_replacements = {
        "default": "zs",
    } if not tot else {
        "default": "sc-zs",
        "spp": "sc-spp",
        "cot": "sc-cot",
    }
    x_replacements = {
        "default": "zs",
    } 

    # {(model, prompt) -> {opponent_type -> total_points}}
    heatmap_data = defaultdict(lambda: defaultdict(list))
    opponent_set = set()

    for model in model_names:
        for prompt in prompt_types:
            for itr in range(iteration_cnt):
                directory = os.path.join(log_dir, f"iteration_{itr}", model, game_type, game_settings_type)

                if not os.path.isdir(directory):
                    continue

                for game_dir in sorted(os.listdir(directory)):
                    info_path = os.path.join(directory, game_dir, 'game.json')
                    if not os.path.isfile(info_path):
                        continue

                    with open(info_path) as f:
                        info = json.load(f)

                    player_types = [info.get(f"player_{i}_player_type") for i in range(2)]
                    if prompt not in player_types:
                        continue

                    model_idx = player_types.index(prompt)
                    if model_idx != 0:
                        continue

                    opponent_type = player_types[1 - model_idx]
                    opponent_set.add(opponent_type)

                    total_points = info.get(f"player_{model_idx}_total_points")
                    if total_points is None:
                        raise ValueError(f"Missing total_points for {info_path}")

                    heatmap_data[(model, prompt)][opponent_type].append(total_points)

    if not heatmap_data:
        raise ValueError("No data collected — check log paths and model+prompt naming conventions.")

    #for key in heatmap_data:
    #    for opponent in heatmap_data[key]:
    #        heatmap_data[key][opponent] /= iteration_cnt

    opponent_types_aux = ["default", "spp", "cot", "srep", "pp", "ap", "tft"]
    sorted_opponents = [opp for opp in opponent_types_aux if opp in opponent_set]
    model_prompt_keys = [(model, prompt) for model in model_names for prompt in prompt_types]

    # Apply x label replacements
    x_labels = sorted_opponents.copy()
    for old, new in x_replacements.items():
        x_labels = [label.replace(old, new) for label in x_labels]

    rows = []
    index_tuples = []

    for model, prompt in model_prompt_keys:
        new_prompt = prompt
        for old, new in y_replacements.items():
            new_prompt = new_prompt.replace(old, new)

        index_tuples.append( (model, new_prompt) )

        values = []
        for opp in sorted_opponents:
            val = heatmap_data.get((model, prompt), {}).get(opp, -1000)
            values.append(val)
        rows.append(values)


    index = pd.MultiIndex.from_tuples(index_tuples, names=["model", "prompt"])
    df = pd.DataFrame(rows, index=index, columns=x_labels)
    return df


In [3]:
dfs_nonsc_tp = [
    get_total_points_dataframe(
        log_dir="../logs/logs_3/data",
        model_names=[model["name"] for model in models],
        prompt_types=prompt_types,
        game_type="rps",
        game_settings_type=game_settings_type,
        iteration_cnt=5,
        tot=False,
    )
    for game_settings_type in game_settings_types
]

dfs_sc_tp = [
    get_total_points_dataframe(
        log_dir="../logs/logs_3/data_tot",
        model_names=[model["name"] for model in models],
        prompt_types=prompt_types,
        game_type="rps",
        game_settings_type=game_settings_type,
        iteration_cnt=2,
        tot=True,
    )
    for game_settings_type in game_settings_types
]


In [4]:
prompt_order = ['zs', 'cot', 'spp', 'sc-zs', 'sc-cot', 'sc-spp']
prompt_order_map = {prompt: i for i, prompt in enumerate(prompt_order)}

dfs_merged_tp = []

for df_nonsc, df_sc in zip(dfs_nonsc_tp, dfs_sc_tp):
    df_nonsc = df_nonsc.copy()
    df_sc = df_sc.copy()

    # Prefix the prompt types in sc
    new_index = []
    for model, prompt in df_sc.index:
        new_index.append((model, prompt))
    df_sc.index = pd.MultiIndex.from_tuples(new_index, names=df_sc.index.names)

    # Concatenate vertically
    merged_df = pd.concat([df_nonsc, df_sc])

    # Reorder by (model, prompt) with custom prompt order
    merged_df = merged_df.reset_index()

    # Add a sort key column
    merged_df['prompt_order'] = merged_df['prompt'].map(prompt_order_map)

    # Sort by model, then prompt order
    merged_df = merged_df.sort_values(['model', 'prompt_order'])

    # Drop helper column
    merged_df = merged_df.drop(columns=['prompt_order'])

    # Restore MultiIndex
    merged_df = merged_df.set_index(['model', 'prompt'])

    dfs_merged_tp.append(merged_df)


In [5]:
import numpy as np

# Define shorter names for the models
rename_map = {
    "Claude 3.5 Sonnet v2": "C3.5Sv2",
    "Claude 3.7 Sonnet": "C3.7S",
    "Claude 3.7 Sonnet (Thinking)": "C3.7S(T)",
    "Claude Sonnet 4": "C4S",
    "Claude Sonnet 4 (Thinking)": "C4S(T)",
    "DeepSeek-R1": "DS-R1",
    "Llama 3.3 70B Instruct": "L3.3-70B",
    "Mistral Large (24.07)": "M-L(24.07)"
}

for df, game_settings_type in zip(dfs_merged_tp, game_settings_types):
    df = df.copy()

    # Rename models to shorter names
    df.rename(index=rename_map, inplace=True)

    # Compute mean & std for each list in df
    mean_df = df.map(lambda x: np.mean(x) if isinstance(x, list) else np.nan)
    std_df  = df.map(lambda x: np.std(x, ddof=1) if isinstance(x, list) else np.nan)

    # Identify maxima based on means
    row_max_mask = mean_df.eq(mean_df.max(axis=1), axis=0)
    col_max_mask = mean_df.eq(mean_df.max(axis=0), axis=1)

    # Prepare styled dataframe with mean ± std
    styled_df = df.copy().astype(str)
    for row in df.index:
        for col in df.columns:
            vals = df.loc[row, col]

            if not isinstance(vals, list) or len(vals) == 0:
                formatted = ""
            else:
                mean_val = np.mean(vals)
                std_val = np.std(vals, ddof=1)  # population std

                # Check if it's a max cell (by mean)
                is_max = row_max_mask.loc[row, col] or col_max_mask.loc[row, col]

                formatted = f"{mean_val:.1f} $\\pm$ {std_val:.1f}"
                if is_max:
                    formatted = f"\\textbf{{{formatted}}}"

            styled_df.loc[row, col] = formatted

    # Add game_settings_type as MultiIndex header
    styled_df.columns = pd.MultiIndex.from_product(
        [[game_settings_type], styled_df.columns],
    )

    latex_code = styled_df.to_latex(
        index=True,
        multicolumn=True,
        multirow=True,
        multicolumn_format='c',
        escape=False,
        caption=f"Total Points Averaged Over All Iterations ({game_settings_type})",
        label=f"tab:rps_total_points_avg_heatmap_{game_settings_type}",
    )

    print(latex_code)
    print("\n\n")


\begin{table}
\caption{Total Points Averaged Over All Iterations (eq1)}
\label{tab:rps_total_points_avg_heatmap_eq1}
\begin{tabular}{lllllllll}
\toprule
 &  & \multicolumn{7}{c}{eq1} \\
 &  & zs & spp & cot & srep & pp & ap & tft \\
model & prompt &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{6}{*}{C3.5Sv2} & zs & 6.6 $\pm$ 13.5 & -1.2 $\pm$ 9.2 & -1.8 $\pm$ 6.9 & -0.4 $\pm$ 3.3 & 6.0 $\pm$ 10.1 & 11.2 $\pm$ 6.4 & \textbf{16.6 $\pm$ 6.9} \\
 & cot & 0.8 $\pm$ 5.4 & 2.0 $\pm$ 5.7 & 5.4 $\pm$ 4.8 & 0.2 $\pm$ 5.3 & 3.8 $\pm$ 11.9 & 6.0 $\pm$ 3.7 & \textbf{13.0 $\pm$ 6.8} \\
 & spp & 8.0 $\pm$ 9.9 & 7.2 $\pm$ 11.6 & 1.6 $\pm$ 6.9 & -1.4 $\pm$ 2.2 & 9.4 $\pm$ 11.2 & 6.8 $\pm$ 1.5 & \textbf{12.8 $\pm$ 7.6} \\
 & sc-zs & -12.5 $\pm$ 12.0 & -0.5 $\pm$ 21.9 & 0.5 $\pm$ 4.9 & -0.5 $\pm$ 9.2 & 1.5 $\pm$ 2.1 & 9.0 $\pm$ 1.4 & \textbf{21.5 $\pm$ 0.7} \\
 & sc-cot & 9.0 $\pm$ 0.0 & -1.0 $\pm$ 2.8 & 0.0 $\pm$ 7.1 & 1.0 $\pm$ 4.2 & 10.5 $\pm$ 14.8 & 14.0 $\pm$ 12.7 & \textbf{16.5 $\pm$ 2.1} \\
 & sc-s

In [6]:
import pandas as pd
import os
import json
from collections import defaultdict

def get_efficiency_dataframe(
    log_dir: str,
    model_names: list[str],
    prompt_types: list[str],
    game_type: str,
    game_settings_type: str,
    iteration_cnt: int,
    tot: bool,
) -> pd.DataFrame:
    y_replacements = {
        "default": "zs",
    } if not tot else {
        "default": "sc-zs",
        "spp": "sc-spp",
        "cot": "sc-cot",
    }
    x_replacements = {
        "default": "zs",
    } 

    # {(model, prompt) -> {opponent_type -> total_points}}
    efficiency_data = defaultdict(lambda: defaultdict(list))
    opponent_set = set()

    for model in model_names:
        for prompt in prompt_types:
            for itr in range(iteration_cnt):
                directory = os.path.join(log_dir, f"iteration_{itr}", model, game_type, game_settings_type)

                if not os.path.isdir(directory):
                    continue

                for game_dir in sorted(os.listdir(directory)):
                    info_path = os.path.join(directory, game_dir, 'game.json')
                    if not os.path.isfile(info_path):
                        continue

                    with open(info_path) as f:
                        info = json.load(f)

                    player_types = [info.get(f"player_{i}_player_type") for i in range(2)]
                    if prompt not in player_types:
                        continue

                    model_idx = player_types.index(prompt)
                    if model_idx != 0:
                        continue

                    opponent_type = player_types[1 - model_idx]
                    opponent_set.add(opponent_type)

                    tokens = info.get(f"player_{model_idx}_tokens")
                    total_points = info.get(f"player_{model_idx}_total_points")

                    if tokens is None or total_points is None:
                        print(f"Model {model}, Prompt {prompt}, Iteration {itr}, Game directory {game_dir} - Missing data in {info_path}")
                        raise ValueError(f"Missing tokens or total_points for {info_path}")

                    efficiency_data[(model, prompt)][opponent_type].append(total_points / max(tokens) * 1000)  # Scale to per 1000 tokens

    if not efficiency_data:
        raise ValueError("No data collected — check log paths and model+prompt naming conventions.")

    #for key in efficiency_data:
    #    for opponent in efficiency_data[key]:
    #        efficiency_data[key][opponent] /= iteration_cnt

    opponent_types_aux = ["default", "spp", "cot", "srep", "pp", "ap", "tft"]
    sorted_opponents = [opp for opp in opponent_types_aux if opp in opponent_set]
    model_prompt_keys = [(model, prompt) for model in model_names for prompt in prompt_types]

    for key in efficiency_data:
        aux = []
        for opp in sorted_opponents:
            aux.append(efficiency_data[key][opp])
        efficiency_data[key]["avg"] = aux

    sorted_opponents.append("avg")  # Add average to the end of the list

    # Apply x label replacements
    x_labels = sorted_opponents.copy()
    for old, new in x_replacements.items():
        x_labels = [label.replace(old, new) for label in x_labels]

    rows = []
    index_tuples = []

    for model, prompt in model_prompt_keys:
        new_prompt = prompt
        for old, new in y_replacements.items():
            new_prompt = new_prompt.replace(old, new)

        index_tuples.append( (model, new_prompt) )

        values = []
        for opp in sorted_opponents:
            val = efficiency_data.get((model, prompt), {}).get(opp, -1000)
            values.append(val)
        rows.append(values)


    index = pd.MultiIndex.from_tuples(index_tuples, names=["model", "prompt"])
    df = pd.DataFrame(rows, index=index, columns=x_labels)
    return df


In [7]:
dfs_nonsc_ef = [
    get_efficiency_dataframe(
        log_dir="../logs/logs_3/data",
        model_names=[model["name"] for model in models],
        prompt_types=prompt_types,
        game_type="rps",
        game_settings_type=game_settings_type,
        iteration_cnt=5,
        tot=False,
    )
    for game_settings_type in game_settings_types
]

dfs_sc_ef = [
    get_efficiency_dataframe(
        log_dir="../logs/logs_3/data_tot",
        model_names=[model["name"] for model in models],
        prompt_types=prompt_types,
        game_type="rps",
        game_settings_type=game_settings_type,
        iteration_cnt=2,
        tot=True,
    )
    for game_settings_type in game_settings_types
]



In [8]:
prompt_order = ['zs', 'cot', 'spp', 'sc-zs', 'sc-cot', 'sc-spp']
prompt_order_map = {prompt: i for i, prompt in enumerate(prompt_order)}

dfs_merged_ef = []

for df_nonsc, df_sc in zip(dfs_nonsc_ef, dfs_sc_ef):
    df_nonsc = df_nonsc.copy()
    df_sc = df_sc.copy()

    # Prefix the prompt types in sc
    new_index = []
    for model, prompt in df_sc.index:
        new_index.append((model, prompt))
    df_sc.index = pd.MultiIndex.from_tuples(new_index, names=df_sc.index.names)

    # Concatenate vertically
    merged_df = pd.concat([df_nonsc, df_sc])

    # Reorder by (model, prompt) with custom prompt order
    merged_df = merged_df.reset_index()

    # Add a sort key column
    merged_df['prompt_order'] = merged_df['prompt'].map(prompt_order_map)

    # Sort by model, then prompt order
    merged_df = merged_df.sort_values(['model', 'prompt_order'])

    # Drop helper column
    merged_df = merged_df.drop(columns=['prompt_order'])

    # Restore MultiIndex
    merged_df = merged_df.set_index(['model', 'prompt'])

    dfs_merged_ef.append(merged_df)


In [9]:
for df, game_settings_type in zip(dfs_merged_ef, game_settings_types):
    df = df.copy()

    # Compute mean & std for each list in df
    mean_df = df.map(lambda x: np.mean(x) if isinstance(x, list) else np.nan)
    std_df  = df.map(lambda x: np.std(x, ddof=1) if isinstance(x, list) else np.nan)

    # Identify maxima based on means
    row_max_mask = mean_df.eq(mean_df.max(axis=1), axis=0)
    col_max_mask = mean_df.eq(mean_df.max(axis=0), axis=1)

    # Prepare styled dataframe with mean ± std
    styled_df = df.copy().astype(str)
    for row in df.index:
        for col in df.columns:
            vals = df.loc[row, col]

            if not isinstance(vals, list) or len(vals) == 0:
                formatted = ""
            else:
                mean_val = np.mean(vals)
                std_val = np.std(vals, ddof=1)  # population std

                # Check if it's a max cell (by mean)
                is_max = row_max_mask.loc[row, col] or col_max_mask.loc[row, col]

                formatted = f"{mean_val:.1f} $\\pm$ {std_val:.1f}"
                if is_max:
                    formatted = f"\\textbf{{{formatted}}}"

            styled_df.loc[row, col] = formatted

    # Add game_settings_type as MultiIndex header
    styled_df.columns = pd.MultiIndex.from_product(
        [[game_settings_type], styled_df.columns],
    )

    # Output LaTeX
    latex_code = styled_df.to_latex(
        index=True,
        multirow=True,
        multicolumn=True,
        multicolumn_format='c',
        escape=False,  # Allow \textbf
        caption=f"Average Efficiency (Points per kilo-token) ({game_settings_type})",
        label=f"tab:rps_efficiency_avg_heatmap_{game_settings_type}",
    )
    print(latex_code)
    print("\n\n")


\begin{table}
\caption{Average Efficiency (Points per kilo-token) (eq1)}
\label{tab:rps_efficiency_avg_heatmap_eq1}
\begin{tabular}{llllllllll}
\toprule
 &  & \multicolumn{8}{c}{eq1} \\
 &  & zs & spp & cot & srep & pp & ap & tft & avg \\
model & prompt &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{6}{*}{Claude 3.5 Sonnet v2} & zs & 1.1 $\pm$ 2.6 & -0.4 $\pm$ 2.0 & -0.4 $\pm$ 1.3 & -0.1 $\pm$ 0.7 & 0.8 $\pm$ 1.4 & 2.5 $\pm$ 2.2 & \textbf{4.1 $\pm$ 2.5} & 1.1 $\pm$ 2.4 \\
 & cot & 0.1 $\pm$ 0.6 & 0.2 $\pm$ 0.6 & 0.6 $\pm$ 0.5 & -0.0 $\pm$ 0.4 & 0.4 $\pm$ 1.2 & 0.6 $\pm$ 0.4 & \textbf{1.3 $\pm$ 0.7} & 0.4 $\pm$ 0.7 \\
 & spp & 0.7 $\pm$ 0.9 & 0.6 $\pm$ 1.0 & 0.1 $\pm$ 0.5 & -0.1 $\pm$ 0.2 & 0.8 $\pm$ 0.9 & 0.5 $\pm$ 0.1 & \textbf{0.9 $\pm$ 0.7} & 0.5 $\pm$ 0.7 \\
 & sc-zs & -0.4 $\pm$ 0.4 & 0.1 $\pm$ 1.2 & 0.0 $\pm$ 0.2 & -0.1 $\pm$ 0.3 & 0.0 $\pm$ 0.1 & 0.4 $\pm$ 0.0 & \textbf{1.2 $\pm$ 0.0} & 0.2 $\pm$ 0.6 \\
 & sc-cot & 0.2 $\pm$ 0.0 & -0.0 $\pm$ 0.1 & -0.0 $\pm$ 0.1 & 0.0 $\pm$ 0.

In [10]:
import pandas as pd
import os
import json
from collections import defaultdict

def get_round_of_understood_opponent(
    log_dir: str,
    model_names: list[str],
    prompt_types: list[str],
    game_type: str,
    game_settings_type: str,
    iteration_cnt: int,
    tot: bool,
) -> pd.DataFrame:
    y_replacements = {
        "default": "zs",
    } if not tot else {
        "default": "sc-zs",
        "spp": "sc-spp",
        "cot": "sc-cot",
    }
    x_replacements = {
        "default": "zs",
    } 

    # {(model, prompt) -> {opponent_type -> round_of_understood list}}
    round_data = defaultdict(lambda: defaultdict(list))
    opponent_set = set()

    for model in model_names:
        for prompt in prompt_types:
            for itr in range(iteration_cnt):
                directory = os.path.join(log_dir, f"iteration_{itr}", model, game_type, game_settings_type)

                if not os.path.isdir(directory):
                    continue

                for game_dir in sorted(os.listdir(directory)):
                    info_path = os.path.join(directory, game_dir, 'game.json')
                    if not os.path.isfile(info_path):
                        continue

                    with open(info_path) as f:
                        info = json.load(f)

                    player_types = [info.get(f"player_{i}_player_type") for i in range(2)]
                    if prompt not in player_types:
                        continue

                    model_idx = player_types.index(prompt)
                    if model_idx != 0:
                        continue

                    opponent_type = player_types[1 - model_idx]
                    opponent_set.add(opponent_type)

                    tokens = info.get(f"player_{model_idx}_tokens")
                    points = info.get(f"player_{model_idx}_points")

                    if tokens is None or points is None:
                        print(f"Model {model}, Prompt {prompt}, Iteration {itr}, Game directory {game_dir} - Missing data in {info_path}")
                        raise ValueError(f"Missing tokens or points for {info_path}")

                    win_rates = []
                    wins = 0
                    for rounds_aux, point in enumerate(reversed(points)):
                        rounds = rounds_aux + 1  # Rounds are 1-indexed in the game
                        if point >= 0:
                            wins += 1
                        win_rate = wins / rounds
                        win_rates.insert(0, win_rate)  # Insert at the beginning to keep order
                    
                    target_percentage = 0.9
                    round_of_understood = next((i + 1 for i, rate in enumerate(win_rates) if rate >= target_percentage), len(win_rates) + 1)
                    
                    round_data[(model, prompt)][opponent_type].append(round_of_understood)


    if not round_data:
        raise ValueError("No data collected — check log paths and model+prompt naming conventions.")

    #for key in round_data:
    #    for opponent in round_data[key]:
    #        round_data[key][opponent] /= iteration_cnt

    opponent_types_aux = ["default", "spp", "cot", "srep", "pp", "ap", "tft"]
    sorted_opponents = [opp for opp in opponent_types_aux if opp in opponent_set]
    model_prompt_keys = [(model, prompt) for model in model_names for prompt in prompt_types]

    # Apply x label replacements
    x_labels = sorted_opponents.copy()
    for old, new in x_replacements.items():
        x_labels = [label.replace(old, new) for label in x_labels]

    rows = []
    index_tuples = []

    for model, prompt in model_prompt_keys:
        new_prompt = prompt
        for old, new in y_replacements.items():
            new_prompt = new_prompt.replace(old, new)

        index_tuples.append( (model, new_prompt) )

        values = []
        for opp in sorted_opponents:
            val = round_data.get((model, prompt), {}).get(opp, -1000)
            values.append(val)
        rows.append(values)


    index = pd.MultiIndex.from_tuples(index_tuples, names=["model", "prompt"])
    df = pd.DataFrame(rows, index=index, columns=x_labels)
    return df


In [11]:
dfs_nonsc_r = [
    get_round_of_understood_opponent(
        log_dir="../logs/logs_3/data",
        model_names=[model["name"] for model in models],
        prompt_types=prompt_types,
        game_type="rps",
        game_settings_type=game_settings_type,
        iteration_cnt=5,
        tot=False,
    )
    for game_settings_type in game_settings_types
]

dfs_sc_r = [
    get_round_of_understood_opponent(
        log_dir="../logs/logs_3/data_tot",
        model_names=[model["name"] for model in models],
        prompt_types=prompt_types,
        game_type="rps",
        game_settings_type=game_settings_type,
        iteration_cnt=2,
        tot=True,
    )
    for game_settings_type in game_settings_types
]



In [12]:
prompt_order = ['zs', 'cot', 'spp', 'sc-zs', 'sc-cot', 'sc-spp']
prompt_order_map = {prompt: i for i, prompt in enumerate(prompt_order)}

dfs_merged_r = []

for df_nonsc, df_sc in zip(dfs_nonsc_r, dfs_sc_r):
    df_nonsc = df_nonsc.copy()
    df_sc = df_sc.copy()

    # Prefix the prompt types in sc
    new_index = []
    for model, prompt in df_sc.index:
        new_index.append((model, prompt))
    df_sc.index = pd.MultiIndex.from_tuples(new_index, names=df_sc.index.names)

    # Concatenate vertically
    merged_df = pd.concat([df_nonsc, df_sc])

    # Reorder by (model, prompt) with custom prompt order
    merged_df = merged_df.reset_index()

    # Add a sort key column
    merged_df['prompt_order'] = merged_df['prompt'].map(prompt_order_map)

    # Sort by model, then prompt order
    merged_df = merged_df.sort_values(['model', 'prompt_order'])

    # Drop helper column
    merged_df = merged_df.drop(columns=['prompt_order'])

    # Restore MultiIndex
    merged_df = merged_df.set_index(['model', 'prompt'])

    dfs_merged_r.append(merged_df)


In [13]:
import numpy as np

# Define shorter names for the models
rename_map = {
    "Claude 3.5 Sonnet v2": "C3.5Sv2",
    "Claude 3.7 Sonnet": "C3.7S",
    "Claude 3.7 Sonnet (Thinking)": "C3.7S(T)",
    "Claude Sonnet 4": "C4S",
    "Claude Sonnet 4 (Thinking)": "C4S(T)",
    "DeepSeek-R1": "DS-R1",
    "Llama 3.3 70B Instruct": "L3.3-70B",
    "Mistral Large (24.07)": "M-L(24.07)"
}

for df, game_settings_type in zip(dfs_merged_r, game_settings_types):
    df = df.copy()

    # Rename models to shorter names
    df.rename(index=rename_map, inplace=True)

    # Compute mean & std for each list in df
    mean_df = df.map(lambda x: np.mean(x) if isinstance(x, list) else np.nan)
    std_df  = df.map(lambda x: np.std(x, ddof=1) if isinstance(x, list) else np.nan)

    # Identify minima based on means
    row_min_mask = mean_df.eq(mean_df.min(axis=1), axis=0)
    col_min_mask = mean_df.eq(mean_df.min(axis=0), axis=1)

    # Make values strings, bold if min in row or column
    styled_df = df.copy().astype(str)
    for row in df.index:
        for col in df.columns:
            vals = df.loc[row, col]

            if not isinstance(vals, list) or len(vals) == 0:
                formatted = ""
            else:
                mean_val = np.mean(vals)
                std_val = np.std(vals, ddof=1)  # population std

                # Check if it's a min cell (by mean)
                is_min = row_min_mask.loc[row, col] or col_min_mask.loc[row, col]

                formatted = f"{mean_val:.1f} $\\pm$ {std_val:.1f}"
                if is_min:
                    formatted = f"\\textbf{{{formatted}}}"

            styled_df.loc[row, col] = formatted

    # Add game_settings_type as MultiIndex header
    styled_df.columns = pd.MultiIndex.from_product(
        [[game_settings_type], styled_df.columns],
    )

    # Output LaTeX
    latex_code = styled_df.to_latex(
        index=True,
        multirow=True,
        multicolumn=True,
        multicolumn_format='c',
        escape=False,  # Allow \textbf
        caption=f"Round \\# where the Agent understood the opponent's Strategy ({game_settings_type})",
        label=f"tab:rps_round_heatmap_{game_settings_type}",
    )
    print(latex_code)
    print("\n\n")


\begin{table}
\caption{Round \# where the Agent understood the opponent's Strategy (eq1)}
\label{tab:rps_round_heatmap_eq1}
\begin{tabular}{lllllllll}
\toprule
 &  & \multicolumn{7}{c}{eq1} \\
 &  & zs & spp & cot & srep & pp & ap & tft \\
model & prompt &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{6}{*}{C3.5Sv2} & zs & 10.6 $\pm$ 13.1 & 21.4 $\pm$ 4.6 & 19.6 $\pm$ 5.6 & 21.0 $\pm$ 3.5 & 14.6 $\pm$ 12.4 & 10.4 $\pm$ 11.2 & \textbf{1.0 $\pm$ 0.0} \\
 & cot & 17.2 $\pm$ 7.3 & 19.6 $\pm$ 7.5 & 20.6 $\pm$ 5.6 & 19.6 $\pm$ 7.1 & 16.0 $\pm$ 11.9 & 22.6 $\pm$ 2.1 & \textbf{5.8 $\pm$ 10.7} \\
 & spp & 11.2 $\pm$ 10.6 & 11.8 $\pm$ 10.9 & 20.4 $\pm$ 5.9 & 23.0 $\pm$ 1.9 & 11.4 $\pm$ 11.3 & 18.4 $\pm$ 9.3 & \textbf{1.6 $\pm$ 0.9} \\
 & sc-zs & 25.0 $\pm$ 0.0 & 14.0 $\pm$ 15.6 & 21.5 $\pm$ 4.9 & 13.5 $\pm$ 16.3 & 12.0 $\pm$ 15.6 & 19.5 $\pm$ 6.4 & \textbf{1.0 $\pm$ 0.0} \\
 & sc-cot & 15.0 $\pm$ 14.1 & 20.5 $\pm$ 0.7 & 22.5 $\pm$ 3.5 & 25.0 $\pm$ 0.0 & 12.5 $\pm$ 16.3 & 7.0 $\pm$ 8.5 & \textbf{1.

In [14]:
# Choose the opponent you want to extract
target_opponent = "avg"

# Assume you have:
# - dfs_merged: list of merged DataFrames (one per game_settings_type)
# - game_settings_types: list of corresponding names

# Collect slices for the target opponent from each df
slices = []

for df, setting in zip(dfs_merged_ef, game_settings_types):
    df = df.copy()
    if target_opponent not in df.columns:
        raise ValueError(f"Opponent '{target_opponent}' not found in columns of {setting}.")
    
    # Series with (model, prompt) index and opponent points as values
    s = df[target_opponent].rename(setting)
    slices.append(s)

# Merge all into one DataFrame on index
result_df = pd.concat(slices, axis=1)


In [15]:
def print_latex_heatmap(df):
    df = df.copy()

    # Identify maxima
    row_max_mask = df.eq(df.max(axis=1), axis=0)
    col_max_mask = df.eq(df.max(axis=0), axis=1)

    # Prepare styled dataframe with mean ± std
    styled_df = df.copy().astype(str)
    for row in df.index:
        for col in df.columns:
            vals = df.loc[row, col]

            if not isinstance(vals, list) or len(vals) == 0:
                formatted = ""
            else:
                mean_val = np.mean(vals)
                std_val = np.std(vals, ddof=1)  # population std

                # Check if it's a max cell (by mean)
                is_max = row_max_mask.loc[row, col] or col_max_mask.loc[row, col]

                formatted = f"{mean_val:.1f} $\\pm$ {std_val:.1f}"
                if is_max:
                    formatted = f"\\textbf{{{formatted}}}"

            styled_df.loc[row, col] = formatted

    # Output LaTeX
    latex_code = styled_df.to_latex(
        index=True,
        multirow=True,
        multicolumn=True,
        multicolumn_format='c',
        escape=False,  # Allow \textbf
        caption="Average Efficiency (Points per kilo-token)",
        label="tab:pd_efficiency_avg_heatmap",
    )
    print(latex_code)
    print("\n\n")

print_latex_heatmap(result_df)

\begin{table}
\caption{Average Efficiency (Points per kilo-token)}
\label{tab:pd_efficiency_avg_heatmap}
\begin{tabular}{llllll}
\toprule
 &  & eq1 & eq1-alt & ba3 & ba3-alt \\
model & prompt &  &  &  &  \\
\midrule
\multirow[t]{6}{*}{Claude 3.5 Sonnet v2} & zs & \textbf{1.1 $\pm$ 2.4} & 0.1 $\pm$ 1.3 & 0.5 $\pm$ 2.7 & 0.3 $\pm$ 1.6 \\
 & cot & 0.4 $\pm$ 0.7 & \textbf{-0.2 $\pm$ 0.8} & 0.5 $\pm$ 1.6 & -0.0 $\pm$ 1.3 \\
 & spp & 0.5 $\pm$ 0.7 & 0.0 $\pm$ 0.5 & \textbf{0.7 $\pm$ 0.8} & 0.1 $\pm$ 1.1 \\
 & sc-zs & 0.2 $\pm$ 0.6 & \textbf{0.1 $\pm$ 0.5} & 0.5 $\pm$ 0.6 & -0.0 $\pm$ 0.4 \\
 & sc-cot & \textbf{0.1 $\pm$ 0.2} & -0.1 $\pm$ 0.2 & 0.1 $\pm$ 0.4 & -0.0 $\pm$ 0.4 \\
 & sc-spp & 0.0 $\pm$ 0.2 & \textbf{-0.0 $\pm$ 0.2} & 0.2 $\pm$ 0.3 & -0.0 $\pm$ 0.2 \\
\cline{1-6}
\multirow[t]{6}{*}{Claude 3.7 Sonnet} & zs & 0.6 $\pm$ 2.4 & 0.2 $\pm$ 2.1 & \textbf{2.0 $\pm$ 4.0} & 0.1 $\pm$ 2.0 \\
 & cot & \textbf{0.7 $\pm$ 0.7} & 0.5 $\pm$ 0.8 & 0.6 $\pm$ 1.1 & 0.7 $\pm$ 1.1 \\
 & spp & 0.5 $\pm$

In [16]:
import pandas as pd
import os
import json
from collections import defaultdict

def get_valid_rate(
    log_dir: str,
    model_names: list[str],
    prompt_types: list[str],
    game_type: str,
    game_settings_type: str,
    iter_cnt: int,
    tot: bool,
) -> pd.DataFrame:
    y_replacements = {
        "zs": "zs",
    } if not tot else {
        "zs": "sc-zs",
        "spp": "sc-spp",
        "cot": "sc-cot",
    }
    x_replacements = {
        "zs": "zs",
    } 

    # {(model, prompt) -> {opponent_type -> valid-rate list}}
    valid_data = defaultdict(lambda: defaultdict(list))
    opponent_set = set()

    for model in model_names:
        for prompt in prompt_types:
            for itr in range(iter_cnt):
                directory = os.path.join(log_dir, f"iteration_{itr}", model, game_type, game_settings_type)

                if not os.path.isdir(directory):
                    continue

                for game_dir in sorted(os.listdir(directory)):
                    info_path = os.path.join(directory, game_dir, 'game.json')
                    if not os.path.isfile(info_path):
                        continue

                    with open(info_path) as f:
                        info = json.load(f)

                    player_types = [info.get(f"player_{i}_player_type") for i in range(2)]
                    if prompt not in player_types:
                        continue

                    model_idx = player_types.index(prompt)
                    if model_idx != 0:
                        continue

                    opponent_type = player_types[1 - model_idx]
                    opponent_set.add(opponent_type)

                    valid_outcomes = info.get("valid_outcomes")

                    if valid_outcomes is None:
                        print(f"Model {model}, Prompt {prompt}, Iteration {itr}, Game directory {game_dir} - Missing data in {info_path}")
                        raise ValueError(f"Missing moves for {info_path}")

                    # find percentage of true valid outcomes
                    total_outcomes = len(valid_outcomes)
                    if total_outcomes == 0:
                        print(f"Model {model}, Prompt {prompt}, Iteration {itr}, Game directory {game_dir} - No valid outcomes in {info_path}")
                        continue

                    valid_count = sum(1 for outcome in valid_outcomes if outcome)
                    valid_rate = valid_count / total_outcomes
                    
                    valid_data[(model, prompt)][opponent_type].append(valid_rate)


    if not valid_data:
        raise ValueError("No data collected — check log paths and model+prompt naming conventions.")

    #for key in valid_data:
    #    for opponent in valid_data[key]:
    #        valid_data[key][opponent] /= iter_cnt
    #        valid_data[key][opponent] *= 100  # Convert to percentage

    opponent_types_aux = ["default", "spp", "cot", "srep", "pp", "mf", "tft"]
    sorted_opponents = [opp for opp in opponent_types_aux if opp in opponent_set]
    model_prompt_keys = [(model, prompt) for model in model_names for prompt in prompt_types]

    # Apply x label replacements
    x_labels = sorted_opponents.copy()
    for old, new in x_replacements.items():
        x_labels = [label.replace(old, new) for label in x_labels]

    rows = []
    index_tuples = []

    for model, prompt in model_prompt_keys:
        new_prompt = prompt
        for old, new in y_replacements.items():
            new_prompt = new_prompt.replace(old, new)

        index_tuples.append( (model, new_prompt) )

        values = []
        for opp in sorted_opponents:
            val = valid_data.get((model, prompt), {}).get(opp, [-1000])
            values.append(val)
        rows.append(values)


    index = pd.MultiIndex.from_tuples(index_tuples, names=["model", "prompt"])
    df = pd.DataFrame(rows, index=index, columns=x_labels)
    return df


In [17]:
dfs_nonsc_v = [
    get_valid_rate(
        log_dir="../logs/logs_3/data",
        model_names=[model["name"] for model in models],
        prompt_types=prompt_types,
        game_type="rps",
        game_settings_type=game_settings_type,
        iter_cnt=5,
        tot=False,
    )
    for game_settings_type in game_settings_types
]

dfs_sc_v = [
    get_valid_rate(
        log_dir="../logs/logs_3/data_tot",
        model_names=[model["name"] for model in models],
        prompt_types=prompt_types,
        game_type="rps",
        game_settings_type=game_settings_type,
        iter_cnt=2,
        tot=True,
    )
    for game_settings_type in game_settings_types
]



In [18]:
dfs_nonsc_v[0]

default  \
model                        prompt                               
Claude 3.5 Sonnet v2         default  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             cot      [1.0, 1.0, 1.0, 1.0, 1.0]   
Claude 3.7 Sonnet            default  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             cot      [1.0, 1.0, 1.0, 1.0, 1.0]   
Claude 3.7 Sonnet (Thinking) default  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             cot      [1.0, 1.0, 1.0, 1.0, 1.0]   
Claude Sonnet 4              default  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             cot      [1.0, 1.0, 1.0, 1.0, 1.0]   
Claude Sonnet 4 (Thinking)   default  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             cot      [1.0, 1.0, 1.0, 1.0, 1.0]   
Llama 3.3 70B Instruct       default  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             cot      [1.0, 1.0, 1.0, 1.0, 1.0]   
Mistral Large (24.07)        default  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             cot      [1.0, 1.0, 1.0, 1.0, 1.0]   
DeepSeek-R1                  default  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             cot      [1.0, 1.0, 1.0, 1.0, 1.0]   

                                                                            spp  \
model                        prompt                                               
Claude 3.5 Sonnet v2         default                  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp                      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             cot                      [1.0, 1.0, 1.0, 1.0, 1.0]   
Claude 3.7 Sonnet            default                  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp                      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             cot                      [1.0, 1.0, 1.0, 1.0, 1.0]   
Claude 3.7 Sonnet (Thinking) default                  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp                      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             cot                      [1.0, 1.0, 1.0, 1.0, 1.0]   
Claude Sonnet 4              default                  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp                      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             cot                    [1.0, 1.0, 1.0, 1.0, 0.875]   
Claude Sonnet 4 (Thinking)   default                  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp                      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             cot                      [1.0, 1.0, 1.0, 1.0, 1.0]   
Llama 3.3 70B Instruct       default                  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp                      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             cot                      [1.0, 1.0, 1.0, 1.0, 1.0]   
Mistral Large (24.07)        default                  [1.0, 1.0, 1.0, 0.0, 1.0]   
                             spp      [1.0, 0.9583333333333334, 0.75, 1.0, 1.0]   
                             cot                      [1.0, 1.0, 1.0, 1.0, 1.0]   
DeepSeek-R1                  default                  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp                      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             cot                      [1.0, 1.0, 1.0, 1.0, 1.0]   

                                                                           cot  \
model                        prompt                                     

In [19]:
prompt_order = ['zs', 'cot', 'spp', 'sc-zs', 'sc-cot', 'sc-spp']
prompt_order_map = {prompt: i for i, prompt in enumerate(prompt_order)}

dfs_merged_v = []

for df_nonsc, df_sc in zip(dfs_nonsc_v, dfs_sc_v):
    df_nonsc = df_nonsc.copy()
    df_sc = df_sc.copy()

    # Prefix the prompt types in sc
    new_index = []
    for model, prompt in df_sc.index:
        new_index.append((model, prompt))
    df_sc.index = pd.MultiIndex.from_tuples(new_index, names=df_sc.index.names)

    # Concatenate vertically
    merged_df = pd.concat([df_nonsc, df_sc])

    # Reorder by (model, prompt) with custom prompt order
    merged_df = merged_df.reset_index()

    # Add a sort key column
    merged_df['prompt_order'] = merged_df['prompt'].map(prompt_order_map)

    # Sort by model, then prompt order
    merged_df = merged_df.sort_values(['model', 'prompt_order'])

    # Drop helper column
    merged_df = merged_df.drop(columns=['prompt_order'])

    # Restore MultiIndex
    merged_df = merged_df.set_index(['model', 'prompt'])

    dfs_merged_v.append(merged_df)


In [20]:
dfs_merged_v[0]

default  \
model                        prompt                               
Claude 3.5 Sonnet v2         cot      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             sc-cot                  [1.0, 1.0]   
                             sc-spp                  [1.0, 1.0]   
                             default  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             default                 [1.0, 1.0]   
Claude 3.7 Sonnet            cot      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             sc-cot                  [1.0, 1.0]   
                             sc-spp                  [1.0, 1.0]   
                             default  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             default                 [1.0, 1.0]   
Claude 3.7 Sonnet (Thinking) cot      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             sc-cot                  [1.0, 1.0]   
                             sc-spp                  [1.0, 1.0]   
                             default  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             default                 [1.0, 1.0]   
Claude Sonnet 4              cot      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             sc-cot                  [1.0, 1.0]   
                             sc-spp                  [1.0, 1.0]   
                             default  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             default                 [1.0, 1.0]   
Claude Sonnet 4 (Thinking)   cot      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             sc-cot                  [1.0, 1.0]   
                             sc-spp                  [1.0, 1.0]   
                             default  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             default                 [1.0, 1.0]   
DeepSeek-R1                  cot      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             sc-cot                  [1.0, 1.0]   
                             sc-spp                  [1.0, 1.0]   
                             default  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             default                 [1.0, 1.0]   
Llama 3.3 70B Instruct       cot      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             sc-cot                  [1.0, 1.0]   
                             sc-spp                  [1.0, 1.0]   
                             default  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             default                 [1.0, 1.0]   
Mistral Large (24.07)        cot      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             sc-cot                  [1.0, 1.0]   
                             sc-spp                  [1.0, 1.0]   
                             default  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             default                 [1.0, 1.0]   

                                                                            spp  \
model                        prompt                                               
Claude 3.5 Sonnet v2         cot                      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             spp                      [1.0, 1.0, 1.0, 1.0, 1.0]   
                             sc-cot                                  [1.0, 1.0]   
                             sc-spp                                  [1.0, 1.0]   
                             default                  [1.0, 1.0, 1.0, 1.0, 1.0]   
                             default                                 [1.0, 1.0]   
Claude 3.7 Sonnet            cot         

In [21]:
model_aggs = {}

for df, game_setting in zip(dfs_merged_v, game_settings_types):
    # df has index (model, prompt) and columns = opponent types

    # Step 1: aggregate across opponents
    df_agg_opponents = df.aggregate(lambda x: [item for sublist in x if isinstance(sublist, list) for item in sublist], axis=1)

    # Step 2: group by model and aggregate across prompts
    df_agg_model = df_agg_opponents.groupby(level=0).aggregate(lambda x: [item for sublist in x if isinstance(sublist, list) for item in sublist])

    model_aggs[game_setting] = df_agg_model

# Combine into a single DataFrame
df_model_agg = pd.DataFrame(model_aggs)




In [ ]:
def print_valid_rates(df):
    df = df.copy()

    # Make values strings, bold if min in row or column
    styled_df = df.copy().astype(str)
    for row in df.index:
        for col in df.columns:
            vals = df.loc[row, col]

            if not isinstance(vals, list) or len(vals) == 0:
                formatted = ""
            else:
                mean_val = np.mean(vals)
                std_val = np.std(vals, ddof=1)  # population std

                formatted = f"{mean_val:.1f} $\\pm$ {std_val:.1f}"

            styled_df.loc[row, col] = formatted

    # Output LaTeX
    latex_code = styled_df.to_latex(
        index=True,
        multirow=True,
        multicolumn=True,
        multicolumn_format='c',
        escape=False,  # Allow \textbf
        caption="Average Valid Rate (\\% of Valid Outcomes)",
        label="tab:rps_valid_rates",
    )
    print(latex_code)
    print("\n\n")

print_valid_rates(df_model_agg)

\begin{table}
\caption{Average Valid Rate (\% of Valid Outcomes)}
\label{tab:rps_valid_rates}
\begin{tabular}{lllll}
\toprule
 & eq1 & eq1-alt & ba3 & ba3-alt \\
model &  &  &  &  \\
\midrule
Claude 3.5 Sonnet v2 & 1.0 $\pm$ 0.0 & 1.0 $\pm$ 0.0 & 1.0 $\pm$ 0.0 & 1.0 $\pm$ 0.0 \\
Claude 3.7 Sonnet & 1.0 $\pm$ 0.0 & 1.0 $\pm$ 0.0 & 1.0 $\pm$ 0.0 & 1.0 $\pm$ 0.0 \\
Claude 3.7 Sonnet (Thinking) & 1.0 $\pm$ 0.0 & 1.0 $\pm$ 0.0 & 1.0 $\pm$ 0.0 & 1.0 $\pm$ 0.0 \\
Claude Sonnet 4 & 1.0 $\pm$ 0.0 & 1.0 $\pm$ 0.0 & 1.0 $\pm$ 0.0 & 1.0 $\pm$ 0.0 \\
Claude Sonnet 4 (Thinking) & 1.0 $\pm$ 0.0 & 1.0 $\pm$ 0.0 & 1.0 $\pm$ 0.0 & 1.0 $\pm$ 0.0 \\
DeepSeek-R1 & 1.0 $\pm$ 0.0 & 1.0 $\pm$ 0.0 & 1.0 $\pm$ 0.0 & 1.0 $\pm$ 0.0 \\
Llama 3.3 70B Instruct & 1.0 $\pm$ 0.0 & 1.0 $\pm$ 0.0 & 1.0 $\pm$ 0.0 & 1.0 $\pm$ 0.0 \\
Mistral Large (24.07) & 1.0 $\pm$ 0.1 & 1.0 $\pm$ 0.1 & 1.0 $\pm$ 0.0 & 1.0 $\pm$ 0.0 \\
\bottomrule
\end{tabular}
\end{table}




